In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from logitlens import LogitLens
import pandas as pd
import matplotlib.pyplot as plt
import ast

In [ ]:
MODEL_NAME = "Tower-Babel/Babel-9B-Chat"
LANGUAGE = "English"
logit_lens = LogitLens(LANGUAGE, MODEL_NAME)
MODEL_NAME = MODEL_NAME.split("/")[-1]
layers_original, rates_original, layers_typo, rates_typo = logit_lens.run(
    path1 = f"/home/hyujang/multilingual-inner-lexicon/data/RQ1/WordIdentity/single_token_splitted_{MODEL_NAME}_{LANGUAGE}_v2.csv",
    path2 = f"/home/hyujang/multilingual-inner-lexicon/data/RQ1/WordIdentity/single_token_typos_{MODEL_NAME}_{LANGUAGE}_v2.csv",
    vis=True,
    save=True
)

In [ ]:
MODEL_NAME = "Tower-Babel/Babel-9B-Chat"
LANGUAGE = "German"
logit_lens = LogitLens(LANGUAGE, MODEL_NAME)
MODEL_NAME = MODEL_NAME.split("/")[-1]
layers_original, rates_original, layers_typo, rates_typo = logit_lens.run(
    path1 = f"/home/hyujang/multilingual-inner-lexicon/data/RQ1/WordIdentity/single_token_splitted_{MODEL_NAME}_{LANGUAGE}_v2.csv",
    path2 = f"/home/hyujang/multilingual-inner-lexicon/data/RQ1/WordIdentity/single_token_typos_{MODEL_NAME}_{LANGUAGE}_v2.csv",
    vis=True,
    save=True
)

In [ ]:
# testing with lm head instea of embedding
MODEL_NAME = "google/gemma-3-12b-it"
LANGUAGE = "English"
logit_lens = LogitLens(LANGUAGE, MODEL_NAME)
MODEL_NAME = MODEL_NAME.split("/")[-1]
layers_original, rates_original, layers_typo, rates_typo = logit_lens.run(
    path1 = f"/home/hyujang/multilingual-inner-lexicon/data/RQ1/WordIdentity/single_token_splitted_{MODEL_NAME}_{LANGUAGE}_v2.csv",
    path2 = f"/home/hyujang/multilingual-inner-lexicon/data/RQ1/WordIdentity/single_token_typos_{MODEL_NAME}_{LANGUAGE}_v2.csv",
    vis=True,
    save=True
)

In [ ]:
MODEL_NAME = "google/gemma-3-12b-it"
LANGUAGE = "English"
logit_lens = LogitLens(LANGUAGE, MODEL_NAME)
MODEL_NAME = MODEL_NAME.split("/")[-1]
layers_original, rates_original, layers_typo, rates_typo = logit_lens.run(
    path1 = f"/home/hyujang/multilingual-inner-lexicon/data/RQ1/WordIdentity/single_token_splitted_{MODEL_NAME}_{LANGUAGE}_v2.csv",
    path2 = f"/home/hyujang/multilingual-inner-lexicon/data/RQ1/WordIdentity/single_token_typos_{MODEL_NAME}_{LANGUAGE}_v2.csv",
    vis=True,
    save=True
)

In [ ]:
MODEL_NAME = "google/gemma-3-12b-it"
LANGUAGE = "German"
logit_lens = LogitLens(LANGUAGE, MODEL_NAME)
MODEL_NAME = MODEL_NAME.split("/")[-1]
layers_original, rates_original, layers_typo, rates_typo = logit_lens.run(
    path1 = f"/home/hyujang/multilingual-inner-lexicon/data/RQ1/WordIdentity/single_token_splitted_{MODEL_NAME}_{LANGUAGE}_v2.csv",
    path2 = f"/home/hyujang/multilingual-inner-lexicon/data/RQ1/WordIdentity/single_token_typos_{MODEL_NAME}_{LANGUAGE}_v2.csv",
    vis=True,
    save=True
)

In [ ]:
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"
LANGUAGE = "English"
logit_lens = LogitLens(LANGUAGE, MODEL_NAME)
MODEL_NAME = MODEL_NAME.split("/")[-1]
layers_original, rates_original, layers_typo, rates_typo = logit_lens.run(
    path1 = f"/home/hyujang/multilingual-inner-lexicon/data/RQ1/WordIdentity/single_token_splitted_{MODEL_NAME}_{LANGUAGE}_v2.csv",
    path2 = f"/home/hyujang/multilingual-inner-lexicon/data/RQ1/WordIdentity/single_token_typos_{MODEL_NAME}_{LANGUAGE}_v2.csv",
    vis=True,
    save=True
)

In [ ]:
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"
LANGUAGE = "German"
logit_lens = LogitLens(LANGUAGE, MODEL_NAME)
MODEL_NAME = MODEL_NAME.split("/")[-1]
layers_original, rates_original, layers_typo, rates_typo = logit_lens.run(
    path1 = f"/home/hyujang/multilingual-inner-lexicon/data/RQ1/WordIdentity/single_token_splitted_{MODEL_NAME}_{LANGUAGE}_v2.csv",
    path2 = f"/home/hyujang/multilingual-inner-lexicon/data/RQ1/WordIdentity/single_token_typos_{MODEL_NAME}_{LANGUAGE}_v2.csv",
    vis=True,
    save=True
)

In [ ]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

# Load Output File and Visualize

In [ ]:
# Define colors for each model
model_colors = {
    "Babel-9B-Chat": "#66c2a5",
    "gemma-3-12b-it": "#fc8d62",
    "Llama-2-7b-chat-hf": "#e78ac3"
}

# Define colors for each language
language_colors = {
    "English": "#1f77b4",
    "German": "#ff7f0e",
    "Korean": "#2ca02c"
}

# Define line styles for each model
model_styles = {
    "Babel-9B-Chat": "-",
    "gemma-3-12b-it": "--",
    "Llama-2-7b-chat-hf": ":"
}

languague_styles = {
    "English": "-",
    "Korean": "--",
    "German": ":"
}

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from logitlens import get_retrieval_rates, get_cumulative_retrieval_rates
import ast

In [ ]:
# Create a grid of subplots (2 rows for languages, 3 columns for models)
fig, axes = plt.subplots(2, 3, figsize=(18, 10), sharex=False, sharey=True)

for col_idx, model_name in enumerate(model_colors.keys()):
    for row_idx, language in enumerate(["English", "German"]):
        # Load data
        results_df_original = pd.read_csv(f"/home/hyujang/multilingual-inner-lexicon/output/RQ1/WordIdentity/single_token_simple_split_{model_name}_{language}_v3.csv")
        results_df_typo = pd.read_csv(f"/home/hyujang/multilingual-inner-lexicon/output/RQ1/WordIdentity/single_token_typo_split_{model_name}_{language}_v3.csv")

        # Convert columns starting with '[' to lists
        for col in results_df_original.columns:
            if results_df_original[col].astype(str).str.startswith("[").any():
                results_df_original[col] = results_df_original[col].apply(ast.literal_eval)
        for col in results_df_typo.columns:
            if results_df_typo[col].astype(str).str.startswith("[").any():
                results_df_typo[col] = results_df_typo[col].apply(ast.literal_eval)

        # Calculate retrieval rates
        layers_original, rates_original = get_retrieval_rates(results_df_original)
        layers_typo, rates_typo = get_retrieval_rates(results_df_typo)

        # Plot on the corresponding subplot
        ax = axes[row_idx, col_idx]
        ax.plot(layers_original, rates_original, marker='o', label='Splits', color="#adaf3d", alpha=0.7)
        ax.plot(layers_typo, rates_typo, marker='o', label='Typo Splits', color="#8c59bb", alpha=0.7)

        # Set title and labels
        if row_idx == 0:
            ax.set_title(model_name, fontsize=22)
        if col_idx == 0:
            ax.set_ylabel(f"{language} Retrieval Rate (Top-3)", fontsize=16)

        ax.set_xlabel("Layer", fontsize=16)
        ax.tick_params(axis='both', which='major', labelsize=14)
        ax.legend(fontsize=14)
        ax.grid(True)

        # Set x-axis range dynamically based on layers
        ax.set_xlim([min(layers_original + layers_typo)-1, max(layers_original + layers_typo)+1])

# Adjust layout
plt.tight_layout()
plt.show()